# Limpeza e tratamento dos dados

#### **Deletar** antes de subir no Github

In [135]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1 Importando csv como dataframe

Carregando bibliotecas necessárias

In [136]:
import pandas as pd
import numpy as np

# caminho para salvamento dos arquivos
caminho = '/content/drive/MyDrive/Colab Notebooks/dados/'

Carregando as bases brutas extraídas

In [137]:
df_consolidado = pd.read_csv(caminho + '/df_consolidado.csv', encoding='latin1')
df_lista_inst = pd.read_csv(caminho + '/df_lista_inst.csv', encoding='latin1')
df_reclamações = pd.read_csv(caminho + '/df_reclamações.csv', encoding='latin1')
df_tarifas = pd.read_csv(caminho + '/df_tarifas.csv', encoding='latin1')

## 2 Data Wrangling

#### Limpando o ``` df_consolidado```



In [138]:
df_consolidado_normal = df_consolidado.copy()

Deletando colunas desnecessárias e renomeando

In [139]:
# deletando colunas desnecessárias
df_consolidado_normal.drop(['Unnamed: 0'], axis=1, inplace=True)

# Listando nomes atuais
print(df_consolidado_normal.columns)

# Renomeando
novos_nomes_colunas = {'Codigo': 'codigo_grupo', 'Nome': 'descr_grupo'}
df_consolidado_normal.rename(columns=novos_nomes_colunas, inplace=True)
df_consolidado_normal.head(2)

Index(['Codigo', 'Nome'], dtype='object')


,codigo_grupo,descr_grupo
0,1,Bancos privados
1,2,Bancos públicos + Caixa Econômica Federal


#### Limpando o ``` df_lista_inst```


In [140]:
df_lista_inst_normal = df_lista_inst.copy()

Deletando colunas desnecessárias e renomeando

In [141]:
# deletando colunas desnecessárias
df_lista_inst_normal.drop(['Unnamed: 0'], axis=1, inplace=True)

# Listando nomes atuais
print(df_lista_inst_normal.columns)

# Renomeando
novos_nomes_colunas = {'Cnpj': 'cnpj', 'Nome': 'nome_instituicao', 'CodigoGrupo': 'codigo_grupo'}
df_lista_inst_normal.rename(columns=novos_nomes_colunas, inplace=True)
df_lista_inst_normal.head(2)

Index(['Cnpj', 'Nome', 'CodigoGrupo'], dtype='object')


,cnpj,nome_instituicao,codigo_grupo
0,14388334,PARANÁ BANCO S.A.,1
1,2801938,BANCO MORGAN STANLEY S.A.,1


#### Limpando o ``` df_reclamações```




In [142]:
df_reclamações_normal = df_reclamações.copy()

Criando chave ``` fk_calendario```



In [143]:
df_reclamações_normal['fk_calendario'] = df_reclamações_normal['Ano'].map(str) + df_reclamações_normal['Trimestre'].str.replace('º', '')

Deletando colunas desnecessárias e renomeando

In [144]:
# deletando colunas desnecessárias
df_reclamações_normal.drop(['Ano', 'Trimestre', 'Unnamed: 14', 'Unnamed: 0', 'Instituição financeira', 'Índice', 'Categoria', 'Tipo'], axis=1, inplace=True)

# Renomeando
novos_nomes_colunas = {
    'CNPJ IF': 'fk_cnpj', 
    'Quantidade de reclamações reguladas procedentes': 'quantidade_reclamacoes_reguladas_procedentes', 
    'Quantidade de reclamações reguladas - outras': 'quantidade_reclamacoes_reguladas_outras',
    'Quantidade de reclamações não reguladas': 'quantidade_reclamacoes_nao_reguladas',
    'Quantidade total de reclamações': 'quantidade_total_reclamacoes',
    'Quantidade total de clientes  CCS e SCR': 'quantidade_total_clientes_ccs_e_scr',
    'Quantidade de clientes  CCS': 'quantidade_clientes_ccs',
    'Quantidade de clientes  SCR': 'quantidade_clientes_scr'
    }

df_reclamações_normal.rename(columns=novos_nomes_colunas, inplace=True)

Mantendo apenas linhas com CNPJ

In [145]:
# Substituindo vazios por NaN
df_reclamações_normal.fk_cnpj.replace(" ", float("NaN"), inplace=True)

# Deletando NaN
df_reclamações_normal.dropna(subset = ['fk_cnpj'], inplace=True)


#### Limpando o ``` df_tarifas```

In [146]:
df_tarifas_normal = df_tarifas.copy()

In [147]:
# deletando colunas desnecessárias
df_tarifas_normal.drop(['Unnamed: 0'], axis=1, inplace=True)

# Listando nomes atuais
print(df_tarifas_normal.columns)

# Renomeando
novos_nomes_colunas = {
    'CodigoServico': 'pk_tarifa',
    'CNPJ':'fk_cnpj',
    'Servico': 'descr_tarifa', 
    'Unidade': 'tipo_tarifa',
    'DataVigencia': 'data_vigencia',
    'ValorMaximo': 'valor_maximo',
    'TipoValor': 'tipo_valor',
    'Periodicidade': 'periodicidade'
    }
df_tarifas_normal.rename(columns=novos_nomes_colunas, inplace=True)
df_tarifas_normal.head(2)

Index(['CodigoServico', 'Servico', 'Unidade', 'DataVigencia', 'ValorMaximo',
       'TipoValor', 'Periodicidade', 'CNPJ'],
      dtype='object')


,pk_tarifa,descr_tarifa,tipo_tarifa,data_vigencia,valor_maximo,tipo_valor,periodicidade,fk_cnpj
0,1101,Confecção de cadastro para início de relaciona...,Por cliente,2008-11-07,500.0,Real,Por evento,14388334
1,1206,CHEQUE - Cheque Administrativo,Por Cheque,2008-04-30,30.0,Real,Por evento,14388334


Criando chave ``` fk_calendario```

In [148]:
# Garantindo data_vigencia como data
df_tarifas_normal['data_vigencia'] = df_tarifas_normal['data_vigencia'].astype('datetime64[ns]')

# Criando chave
df_tarifas_normal['fk_calendario'] = df_tarifas_normal.data_vigencia.dt.year.map(str) + df_tarifas_normal.data_vigencia.dt.quarter.map(str)

## 3 Salvando dados tratados

In [149]:
# refazendo cópias

df_consolidado = df_consolidado_normal.copy()
df_lista_inst = df_lista_inst_normal.copy()
df_reclamações = df_reclamações_normal.copy()
df_tarifas = df_tarifas_normal.copy()

In [150]:
df_tarifas.to_csv(caminho + 'normalizadas/df_tarifas.csv', encoding='latin1', index=False)
df_lista_inst.to_csv(caminho + 'normalizadas/df_lista_inst.csv', encoding='latin1', index=False)
df_consolidado.to_csv(caminho + 'normalizadas/df_consolidado.csv', encoding='latin1', index=False)
df_reclamações.to_csv(caminho + 'normalizadas/df_reclamações.csv', encoding='latin1', index=False)